# How to propagate config

:::info Prerequisites

This guide assumes familiarity with the following concepts:

- [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language)
- [LangChain RunnableConfig](/docs/concepts/#runnable-config)
:::


The [RunnableConfig](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.config.RunnableConfig.html) dict contains run time configuration information that needs to be properly
propagated throughout the entire chain. This config contains things like `callbacks` and user provided `tags` and `metadata`.

If you're composing a chain entirely using LCEL, then `RunnableConfig` will be propagated automatically on your behalf.

If you're using `RunnableLambda`, `RunnableGenerator` or `@tool` to create custom components, `langchain` will attempt to propagate callbacks on your behalf from those components to any child `Runnables` when possible.

:::{.callout-important}

If you're working in an `async` code base and are using `python<=3.10`, you will need to propagate `config` (or `callbacks`) manually since LangChain is unable to do this automatically.

:::

## Sync (All Python Versions)

You should see that the custom callback is invoked twice!

In [1]:
import asyncio
from typing import Any, Dict, List

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.runnables import RunnableLambda


class MyCustomHandler(BaseCallbackHandler):
    """Sync callback handler that can be used to handle callbacks from langchain."""

    def on_chain_start(
        self,
        serialized: Dict[str, Any],
        inputs: Dict[str, Any],
        **kwargs: Any,
    ) -> None:
        """Run when chain starts running."""
        print("---")
        print(f"Invoking custom callback. Received inputs: {inputs}")


@RunnableLambda
def foo(
    inputs, config
):  # You don't need to have config. Using it just to print it out.
    print("---")
    print(f"Inspect the config given to foo: `{config}`")
    return "world"


@RunnableLambda
def bar(inputs):
    return foo.invoke("goodbye")


bar.invoke("hello", {"callbacks": [MyCustomHandler()], "tags": ["hello"]});

---
Invoking custom callback. Received inputs: hello
---
Invoking custom callback. Received inputs: goodbye
---
Inspect the config given to foo: `{'tags': ['hello'], 'metadata': {}, 'callbacks': <langchain_core.callbacks.manager.CallbackManager object at 0x7171772f1b90>, 'recursion_limit': 25}`


## Async Python >=3.11

The custom callback will be invoked twice!

This code will work correctly even thought `bar` does not propagate config to `foo` does not propagate config!

In [2]:
import sys

sys.version

'3.11.4 (main, Sep 25 2023, 10:06:23) [GCC 11.4.0]'

In [3]:
import asyncio
from typing import Any, Dict, List

from langchain_core.callbacks import AsyncCallbackHandler
from langchain_core.runnables import RunnableLambda


class AsyncMyCustomHandler(AsyncCallbackHandler):
    """Async callback handler that can be used to handle callbacks from langchain."""

    async def on_chain_start(
        self,
        serialized: Dict[str, Any],
        inputs: Dict[str, Any],
        **kwargs: Any,
    ) -> None:
        """Run when chain starts running."""
        print("---")
        print(f"Invoking custom callback. Received inputs: {inputs}")


@RunnableLambda
async def foo(inputs, config):
    print("---")
    print(f"Inspect the config given to foo: `{config}`")
    return "world"


@RunnableLambda
async def bar(inputs):
    return await foo.ainvoke("goodbye")


await bar.ainvoke("hello", {"callbacks": [AsyncMyCustomHandler()]})

---
Invoking custom callback. Received inputs: hello
---
Invoking custom callback. Received inputs: goodbye
---
Inspect the config given to foo: `{'tags': [], 'metadata': {}, 'callbacks': <langchain_core.callbacks.manager.AsyncCallbackManager object at 0x7171772feb50>, 'recursion_limit': 25}`


'world'

## Async Python <=3.10

### Incorrect

This code will not work!

In [1]:
import sys

sys.version

'3.9.6 (default, Sep 26 2023, 21:46:56) \n[GCC 11.4.0]'

In [2]:
import asyncio
from typing import Any, Dict, List

from langchain_core.callbacks import AsyncCallbackHandler
from langchain_core.runnables import RunnableLambda


class AsyncMyCustomHandler(AsyncCallbackHandler):
    """Async callback handler that can be used to handle callbacks from langchain."""

    async def on_chain_start(
        self,
        serialized: Dict[str, Any],
        inputs: Dict[str, Any],
        **kwargs: Any,
    ) -> None:
        """Run when chain starts running."""
        print("---")
        print(f"Invoking custom callback. Recevied inputs: {inputs}")


@RunnableLambda
async def foo(inputs, config):
    print("---")
    print(f"Inspect the config given to foo: `{config}`")
    return "world"


@RunnableLambda
async def bar(inputs):
    return await foo.ainvoke("goodbye")


await bar.ainvoke("hello", {"callbacks": [AsyncMyCustomHandler()]})

---
Invoking custom callback. Recevied inputs: hello
---
Inspect the config given to foo: `{'tags': [], 'metadata': {}, 'callbacks': <langchain_core.callbacks.manager.AsyncCallbackManager object at 0x76f8f69eb700>, 'recursion_limit': 25}`


'world'

### Correct

This code will work correctly across all supported versions of python.

:::{.callout-tip}

This code exposes `config` and propagates it to child component as `foo.ainvoke('goodbye', config=config)`
:::

In [3]:
@RunnableLambda
async def foo(inputs, config):
    print("---")
    print(f"Inspect the config given to foo: `{config}`")
    return "world"


@RunnableLambda
async def bar(inputs, config):
    return await foo.ainvoke("goodbye", config=config)


await bar.ainvoke("hello", {"callbacks": [AsyncMyCustomHandler()], "tags": ["meow"]})

---
Invoking custom callback. Recevied inputs: hello
---
Invoking custom callback. Recevied inputs: goodbye
---
Inspect the config given to foo: `{'tags': ['meow'], 'metadata': {}, 'callbacks': <langchain_core.callbacks.manager.AsyncCallbackManager object at 0x76f90c796d00>, 'recursion_limit': 25}`


'world'